In [ ]:
using Pkg; Pkg.activate("..")

In [ ]:
using Revise
using CreditRisk
using Distributions
using PyPlot
import Random

### Plotting inner level objective function

In [ ]:
# n = 2000
# c = 2
# s = 2
# l = 0.3
# param = Parameter(n,c,s,l)
N = 1000  # m
C = 2     # 2
S = 2     # d
l = 0.3   # q

cmm = zeros(N,C)
p = 0.05
cmm[:,1] .= p
cmm[:,2] .= 1-p

ead = zeros(N) .+ 1
lgc = zeros(N,C)
lgc[:,1] .= 1
lgc[:,2] .= 0
cn = fill(2,N)

β = zeros(N,C)
β[1:2:N,:] .= [0.7 0]
β[2:2:N,:] .= [0 0.65]
denom = @. sqrt(1 - $sum((β).^2, dims=2))
denom = dropdims(denom; dims=2)

# H[n, c] = inverse_unit_Gaussian(∑ᵧ cmm[c(n), γ])
cum_cmm = cumsum(cmm, dims=2)
H = invnormcdf.(cum_cmm)

weights = zeros(N,C)
weights = ead ./ sum(ead)
weights = weights .* lgc

param = Parameter(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights)

In [ ]:
(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

Ψ = init_Ψ()

Zdist = MvNormal(S, 1)
Z = zeros(S)
pnc = zeros(N, C)
phi0 = zeros(N, C+1)
phi  = @view phi0[:,2:end]
twist = zeros(N, C)
mgf = zeros(N)
qnc = zeros(N, C)

Random.rand!(Zdist, Z)
@. phi = normcdf((H - $(β*Z)) / denom)
diff!(pnc, phi0; dims=2)

objective(θ) = begin
    θ = θ[1]
    Ψ(θ, pnc, weights) - θ*l
end

xs = -10000:10000
ys = [objective([x]) for x in xs]
display(plot(xs, ys))

### Plotting outer level objective function

In [ ]:
s = 2         
range = 5     # [-range, range]
step = 0.1
tails = [0.2]

for tail in tails
    param = Parameter(n,c,s,tail)
    (N, C, S, loss, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

    phi0 = zeros(N, C+1)
    phi  = @view phi0[:,2:end]
    pnc = zeros(N, C)

    Ψ = init_Ψ()
    innerlevel = InnerLevelTwisting(N, C)

    objective(z) = begin
        @. phi = normcdf((H - $(β*z)) / denom)
        diff!(pnc, phi0; dims=2)
        twist!(innerlevel, pnc, weights, loss)
        θ = get_result(innerlevel)
        θ*l - Ψ(θ, pnc, weights) + 0.5z'z
    end

    xs = -range:step:range
    ys = -range:step:range
    zs = [objective([x, y]) for x in xs, y in ys]

    display(surf(xs, ys, zs))
#     savefig("surf_$loss.pdf", format="pdf")
end